# GPU Check

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
for gpu in GPUs:
  print(gpu.name, gpu.memoryTotal)

# Imports

In [ ]:
# from search_eval.utils.common_utils import *
from search_eval.eval_no_search_SGLD import Eval_SGLD, SingleImageDataset

from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader
from search_space.unet.unet import UNet

import numpy as np
import torch
torch.cuda.empty_cache()

dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))


# Execute -- SGLD (No Search)

In [ ]:
# INPUTS
total_iterations = 1200 # optimal 1400
show_every = 100 # 200
report_every = 25 # 25
burnin_iter = 200 # 200
lr = 0.1 #  note a smaller learning rate affecs the SGLD, so overfitting happens FASTER at LOWER learning rates (start with 0.01)

# model = UNet(in_channels=1, out_channels=1, init_features=64, depth=4)
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=1, out_channels=1, init_features=64, pretrained=False)

img_id = np.random.randint(0, 50)
resolution = 64
noise_type = 'gaussian'
noise_level = '0.09'
phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{img_id}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{img_id}.npy')

print(f"\n\n----------------------------------")
print(f'Experiment Configuration:')
print(f'\tImage Config')
print(f'\t\tImage ID: {img_id}')
print(f'\t\tImage Resolution: {resolution}')
print(f'\t\tNoise Type: {noise_type}')
print(f'\t\tNoise Level: {noise_level}\n')

print(f'\tModel Config')
print(f'\t\tModel: {model}\n')

print(f'\tTraining Config')
print(f'\t\tTotal Iterations: {total_iterations}')
print(f'\t\tBurnin Iterations: {burnin_iter}')
print(f'\t\tLearning Rate: {lr}')
print(f'\t\tImage Resolution: {resolution}')
print(f'\t\tPlotting every {show_every} iterations\n')
print(f"----------------------------------\n\n")

# Create the lightning module
module = Eval_SGLD(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                lr=lr, 
                burnin_iter=burnin_iter,
                model=model, # model defaults to U-net 
                show_every=show_every,
                report_every=report_every,
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=None)
lightning.fit(model)